In [1]:
import pandas as pd; import numpy as np; import janitor as jn


In [2]:
pd.__version__

'2.2.1'

In [3]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 'value_1': [2,5,7,1,3,4]})
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 'value_2A': [0,3,7,12,0,2,3,1], 'value_2B': [1,5,9,15,pd.NA,4,6,3]})
df2 = df2.astype({'value_2B':'Int32'})


In [4]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [5]:
events = pd.read_csv('/Users/samuel.oranyeli/Downloads/results.csv', parse_dates=['start','end']).iloc[:, 1:]

In [6]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


19.9 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

19.1 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2],
       keyy = range(1,8)
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15],
       keyy=range(8,0,-1)
       )
D=pd.DataFrame(D)



In [9]:
C = pd.concat([C]*40*40)
D = pd.concat([D]*40*40)

In [10]:
%%timeit
(C
.conditional_join(
    D, 
    
    ('profit','profit','>='),
    # ('keyy','keyy','=='),
  # ('unitsSold','unitsSold','>='),
  ('vol','vol','<='),
  # ('vol','profit','>'),
    use_numba=True)
)

1.53 s ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
(C
.conditional_join(
    D, 
    
    ('profit','profit','>='),
    # ('keyy','keyy','=='),
  # ('unitsSold','unitsSold','>='),
  ('vol','vol','<='),
  # ('vol','profit','>'),
    use_numba=False)
)

1.4 s ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
from string import ascii_lowercase
np.random.seed(1)
# n = 20; k = 20
n = 20_0_000; k = 20_0
mapp = {k:v for v,k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high = 2_00, size = n)
idx2 = np.random.randint(0, high = 3_00, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2),
          )


d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [13]:
%%timeit
bla = (d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=True)
)



283 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
bla = (d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=False)
)

601 ms ± 519 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
employees = pd.read_csv('/Users/samuel.oranyeli/employees.csv')

In [16]:
%%timeit
(employees
.conditional_join(
    employees,
    ('salary', 'salary', '<'),
    ('tax', 'tax', '>'),
    use_numba = True)
)

6.57 s ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
(employees
.conditional_join(
    employees,
    ('salary', 'salary', '<'),
    ('tax', 'tax', '>'),
    use_numba = False)
)

4.02 s ± 8.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
